# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import os
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()
# Import API key
geoapify_key = os.getenv("geoapify_key")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,anadyr,64.7500,177.4833,-1.72,93,75,1.00,RU,2023-11-01
1,1,adamstown,-25.0660,-130.1015,22.89,73,1,7.04,PN,2023-11-01
2,2,hamilton,39.1834,-84.5333,5.95,59,5,0.45,US,2023-11-01
3,3,serebryanyye prudy,54.4692,38.7210,12.55,89,94,3.87,RU,2023-11-01
4,4,ola,59.5833,151.2833,1.19,95,100,4.81,RU,2023-11-01


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = city_data_df["City"],
    s=city_data_df["Humidity"]
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_color,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
%%capture --no-display
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_filter=city_data_df[(city_data_df["Cloudiness"]==0) &
                                 (city_data_df["Max Temp"]<(27)) &
                                 (city_data_df["Max Temp"]>(21)) &
                                 (city_data_df["Wind Speed"]<4.5)]

# Drop any rows with null values
city_data_df_filter

# Display sample data
map_plot_2 = city_data_df_filter.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = city_data_df_filter["City"],
    s=city_data_df_filter["Humidity"],
    frame_width = 700,
    frame_height = 500 
)

map_plot_2
# city_data_df_filter

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_color,_size)

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_filter[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
52,port hedland,AU,-20.3167,118.5667,73,
157,smara,EH,26.7384,-11.6719,65,
188,tindouf,DZ,27.6711,-8.1474,25,
275,harij,IN,23.7000,71.9000,40,
318,songadh,IN,21.1667,73.5667,50,
392,karratha,AU,-20.7377,116.8463,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 1
radius = 10000
categories = "accommodation"
filters = [f"circle:{hotel_df['Lng'].iloc[x]},{hotel_df['Lat'].iloc[x]},{radius}" for x in range(len(hotel_df['Lng']))]
bias = [f"proximity:{hotel_df['Lng'].iloc[x]},{hotel_df['Lat'].iloc[x]}" for x in range(len(hotel_df['Lng']))]
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key,
    "categories":categories
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
name_address={}
for index, row in hotel_df.reset_index().iterrows():

    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    param={"filter":params["filter"][index],
           "limit":params["limit"],
           "bias":params["bias"][index],
           "apiKey":params["apiKey"],
           "categories":params["categories"]
          }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address[index] = requests.get(base_url, params=param).json()
    
    # Convert the API response to JSON format
#     name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[hotel_df.reset_index().iloc[index,0], "Hotel Name"] = name_address[index]["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[hotel_df.reset_index().iloc[index,0], "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.reset_index().loc[index, 'City']} - nearest hotel: {hotel_df.reset_index().loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df


Starting hotel search
port hedland - nearest hotel: The Esplanade Hotel
smara - nearest hotel: Hôtel Amine فندق الأمين
tindouf - nearest hotel: محمد بوسبي
harij - nearest hotel: No hotel found
songadh - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
52,port hedland,AU,-20.3167,118.5667,73,The Esplanade Hotel
157,smara,EH,26.7384,-11.6719,65,Hôtel Amine فندق الأمين
188,tindouf,DZ,27.6711,-8.1474,25,محمد بوسبي
275,harij,IN,23.7000,71.9000,40,No hotel found
318,songadh,IN,21.1667,73.5667,50,No hotel found
392,karratha,AU,-20.7377,116.8463,40,Karratha International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Humidity", "City", "Country"],
    color = city_data_df_filter["City"],
    s=city_data_df_filter["Humidity"],
    frame_width = 700,
    frame_height = 500
)
# Display the map
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_color,_size,Hotel Name,Humidity,City,Country)